> miniconda版本:  https://mirrors.tuna.tsinghua.edu.cn/anaconda/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh

#  1. 产生tick日志

#####  编译最小main : 文件 mini_main.c
```cpp
//文件 mini_main.c
int main(int argc, char** argv){
    return 0;
}
```

```bash
tick_save=true /pubx/build-llvm15/bin/clang-15 -c mini_main.c
#生成tick日志文件:
#-rw-rw-r--  138M  8月  8 22:23 clang-15_960162_1691504630415_1
```

# 2. 加载tick日志

In [1]:
# %config InlineBackend.figure_format = 'svg'
%config InlineBackend.rc={'figure.figsize': (30,20)}

In [2]:
import numpy 
import math

In [3]:
import seaborn

In [4]:
import plotly.offline as plotly_offline
plotly_offline.init_notebook_mode(connected=True)          
import plotly.graph_objs as graph_objs                
import plotly.figure_factory as figure_factory 

from plotly.graph_objs import Scatter, Figure, Scatter3d

import matplotlib.pyplot



In [5]:
import pandas
# pandas.options.display.max_columns = None
# pandas.options.display.max_rows = None




In [6]:
%%bash
ls -lhrt /tick_data_home/ | tail -n 1

-rw-rw-r-- 1 zz zz 138M  8月  8 22:23 clang-15_960162_1691504630415_1


In [7]:
#编译正常: mini_main.c
df=pandas.read_csv(filepath_or_buffer="/tick_data_home/clang-15_960162_1691504630415_1",sep=',' , quotechar="'")


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014494 entries, 0 to 1014493
Data columns (total 18 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   滴答           1014494 non-null  int64 
 1   tickKind     1014494 non-null  int64 
 2   funcEnterId  1014494 non-null  int64 
 3   rTSVarC      1014494 non-null  int64 
 4   d栈生          1014494 non-null  int64 
 5   d栈死          1014494 non-null  int64 
 6   d堆生          1014494 non-null  int64 
 7   d堆死          1014494 non-null  int64 
 8   栈生           1014494 non-null  int64 
 9   栈死           1014494 non-null  int64 
 10  栈净           1014494 non-null  int64 
 11  堆生           1014494 non-null  int64 
 12  堆死           1014494 non-null  int64 
 13  堆净           1014494 non-null  int64 
 14  srcFile      1014494 non-null  object
 15  funcLine     1014494 non-null  int64 
 16  funcCol      1014494 non-null  int64 
 17  funcName     1003994 non-null  object
dtypes: int64(16), object(2

In [9]:
df.tail()

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
1014489,1012462,0,122004,1,1,0,0,0,239876,235690,4186,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
1014490,1012463,0,122004,1,0,0,0,0,239876,235690,4186,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
1014491,1012464,2,122004,1,0,1,0,0,239876,235690,4186,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,195,39,printForSigInfoIfNeeded
1014492,1012465,2,122003,0,0,0,0,0,239876,235691,4185,0,0,0,/pubx/llvm-project/llvm/lib/Support/PrettyStac...,227,49,llvm::PrettyStackTraceEntry::~PrettyStackTrace...
1014493,1011967,1,121841,0,0,0,0,0,239875,235690,4185,0,0,0,/pubx/llvm-project/llvm/lib/Support/MemAlloc.cpp,24,72,llvm::deallocate_buffer


# 3.  过滤出只有 函数进入 没有 函数返回 的tick日志

```cpp
// 文件: http://192.168.1.4:10880/analyze_code/clang-ctk/src/75700ddaacb8f7c1e19be8cabcafefe2831bd5b5/t_clock_tick/t_clock_tick.cpp

enum TickKind{
    //正常tick
    NormalTick=0,
    //函数进入tick 只作为 和 函数返回tick 做比对，看哪里少插入了X__funcReturn
    FuncEnter=1,
    //函数返回tick
    FuncReturn=2

};
```

In [10]:
#函数进入tick
FuncEnter=1
#函数返回tick
FuncReturn=2

tickKind_Text_map={FuncEnter:"函数进入", FuncReturn:"函数返回" }


##  3.1  函数进入tick 或 函数返回tick 的日志行

In [11]:
#函数进入tick 或 函数返回tick 的日志行
df_ER = df[df['tickKind'].isin( [FuncEnter, FuncReturn] )]
df_ER.shape

(241939, 18)

In [12]:
df_ER.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 241939 entries, 0 to 1014493
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   滴答           241939 non-null  int64 
 1   tickKind     241939 non-null  int64 
 2   funcEnterId  241939 non-null  int64 
 3   rTSVarC      241939 non-null  int64 
 4   d栈生          241939 non-null  int64 
 5   d栈死          241939 non-null  int64 
 6   d堆生          241939 non-null  int64 
 7   d堆死          241939 non-null  int64 
 8   栈生           241939 non-null  int64 
 9   栈死           241939 non-null  int64 
 10  栈净           241939 non-null  int64 
 11  堆生           241939 non-null  int64 
 12  堆死           241939 non-null  int64 
 13  堆净           241939 non-null  int64 
 14  srcFile      241939 non-null  object
 15  funcLine     241939 non-null  int64 
 16  funcCol      241939 non-null  int64 
 17  funcName     232324 non-null  object
dtypes: int64(16), object(2)
memory usage: 35.1+

## 3.2 函数进入tick行数, 函数返回tick行数

In [13]:
# 按照tickKind字段分组，并获取每个组的行数
grouped_df_ER = df_ER.groupby('tickKind').size().reset_index()

# 将列名设置为"tickKind"和"行数"
grouped_df_ER.columns = ['tickKind', '该tickKind组内tick行数']

# 创建一个新的DataFrame，将"tickKind"和"行数"作为列添加到新的DataFrame中
g_df_ER = pandas.DataFrame(grouped_df_ER)


print(f"tickKind 枚举值 如下，\n  {tickKind_Text_map} ")

g_df_ER


tickKind 枚举值 如下，
  {1: '函数进入', 2: '函数返回'} 


,tickKind,该tickKind组内tick行数
0,1,122006
1,2,119933


In [14]:
g_df_ER阅读=g_df_ER.copy(deep=True)
g_df_ER阅读['tickKind名']= g_df_ER阅读['tickKind'].apply(lambda x: tickKind_Text_map[x])
g_df_ER阅读

,tickKind,该tickKind组内tick行数,tickKind名
0,1,122006,函数进入
1,2,119933,函数返回


## 3.2B 函数进入但无返回次数

In [15]:
函数进入次数=g_df_ER[g_df_ER.tickKind==FuncEnter].该tickKind组内tick行数.values[0]
函数返回次数=g_df_ER[g_df_ER.tickKind==FuncReturn].该tickKind组内tick行数.values[0] 


函数进入但无返回次数=函数进入次数 - 函数返回次数

print(f"函数进入但无返回次数:{函数进入但无返回次数},占比:{函数进入但无返回次数/函数进入次数}")

函数进入但无返回次数:2073,占比:0.01699096765732833


## 3.3  存在   函数进入 缺少 对应 函数返回

> 3.3.1 、 3.3.2 、3.3.3  是  正确性验证:

> 整个tick日志文件中只保留函数进入、函数返回 即 df_ER, 

> 在df_ER 中, 有以下结论成立:

1. 一个函数进入id只能出现一次，即一个函数进入id 充分且必要地 只表示一次函数调用
2. 当只有一个函数进入id日志时，必然是 函数进入，而不可能是函数返回。 此结论成立，说明 libCTk.so 写的还行。
3. 正常情况下， 一个函数进入id有两条日志，一条 函数进入  ，另一条是 函数返回。 
>  注意 函数返回日志 中 也有 字段 函数进入id, 为的就是表示此次 函数返回 是 因 该函数进入 才返回的。

> 3.3.1 、 3.3.2 说明的是  结论1 成立

> 3.3.3  说明的是 结论2 成立

> 结论3 应该不用验证

### 3.3.1 其内只有一行日志的函数进入id们

In [16]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
_df_ER_1 = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame
df_ER_1 = _df_ER_1.reset_index(drop=True)

df_ER_1.shape

(2221, 18)

In [17]:
df_ER_1.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,196768,1,22926,0,0,0,0,0,55421,55412,9,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of
1,196827,1,22929,0,0,0,0,0,55440,55428,12,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,301,69,llvm::StringRef::find_last_not_of
2,196999,1,22945,0,0,0,0,0,55486,55461,25,0,0,0,/pubx/llvm-project/llvm/lib/Option/OptTable.cpp,404,75,llvm::opt::OptTable::ParseOneArg
3,197010,1,22947,0,0,0,0,0,55491,55461,30,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,260,70,llvm::StringRef::find_first_not_of
4,198126,1,23160,0,0,0,0,0,55731,55706,25,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of


### 3.3.2 其内只有一行日志的函数进入id们  都是 函数进入

In [18]:
_df_ER_1_enter = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1 and x.tickKind == FuncEnter)

df_ER_1_enter = _df_ER_1_enter.reset_index(drop=True)

df_ER_1_enter.shape

(2221, 18)

In [19]:
df_ER_1_enter.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,196768,1,22926,0,0,0,0,0,55421,55412,9,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of
1,196827,1,22929,0,0,0,0,0,55440,55428,12,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,301,69,llvm::StringRef::find_last_not_of
2,196999,1,22945,0,0,0,0,0,55486,55461,25,0,0,0,/pubx/llvm-project/llvm/lib/Option/OptTable.cpp,404,75,llvm::opt::OptTable::ParseOneArg
3,197010,1,22947,0,0,0,0,0,55491,55461,30,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,260,70,llvm::StringRef::find_first_not_of
4,198126,1,23160,0,0,0,0,0,55731,55706,25,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of


### 3.3.3 其内只有一行日志的函数进入id们  都不是 函数返回

In [20]:
_df_ER_k1_return = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1 and x.tickKind == FuncReturn)

df_ER_k1_return = _df_ER_k1_return.reset_index(drop=True)

df_ER_k1_return.shape

(0, 18)

In [21]:
df_ER_k1_return.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName


### 3.3.4  形象化: 函数进入 缺少 对应 函数返回

In [22]:
df_ER_1.head(5)

,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
0,196768,1,22926,0,0,0,0,0,55421,55412,9,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of
1,196827,1,22929,0,0,0,0,0,55440,55428,12,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,301,69,llvm::StringRef::find_last_not_of
2,196999,1,22945,0,0,0,0,0,55486,55461,25,0,0,0,/pubx/llvm-project/llvm/lib/Option/OptTable.cpp,404,75,llvm::opt::OptTable::ParseOneArg
3,197010,1,22947,0,0,0,0,0,55491,55461,30,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,260,70,llvm::StringRef::find_first_not_of
4,198126,1,23160,0,0,0,0,0,55731,55706,25,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of


In [23]:
例子函数进入id=df_ER_1.funcEnterId.values[0]
例子函数进入id

22926

In [24]:
df[ df['funcEnterId'].isin( [例子函数进入id] ) &  df['tickKind'].isin( [FuncEnter, FuncReturn] ) ]


,滴答,tickKind,funcEnterId,rTSVarC,d栈生,d栈死,d堆生,d堆死,栈生,栈死,栈净,堆生,堆死,堆净,srcFile,funcLine,funcCol,funcName
197161,196768,1,22926,0,0,0,0,0,55421,55412,9,0,0,0,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,llvm::StringRef::find_last_of



> 
>  人工检查/pubx/llvm-project/llvm/lib/Support/regcomp.c	 中的 此行列位置，发现 主要是 不带花括号的if return造成的。

```cpp
if(...)  //没有花括号的if
    return;//紧挨着就return

//这种语句，被我人为跳过了，因此要加花括号，才能在if和return之间插入语句。

```

## 3.4  哪些函数 只有   函数进入  却没有  函数返回 

In [25]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
df_ER_k1__ = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame df_ER_k1
df_ER_k1_ = df_ER_k1__.reset_index(drop=True)

# 对df_ER_k1按funcName字段分组，并输出各不同的funcName
只进不出函数名 = df_ER_k1_.groupby('funcName').size().reset_index()
只进不出函数名.columns = ['funcName', '进入次数']

只进不出函数名_进入次数倒序=只进不出函数名.sort_values(by='进入次数', ascending=False)

# pandas.set_option('display.max_rows', None)
# pandas.set_option('display.max_colwidth', None)


只进不出函数名_进入次数倒序 = 只进不出函数名_进入次数倒序.reset_index(drop=True)
只进不出函数名_进入次数倒序.shape


(77, 2)

In [26]:
只进不出函数名_进入次数倒序.head(5)

,funcName,进入次数
0,clang::IdentifierInfo::getPPKeywordID,748
1,clang::Preprocessor::HandleDirective,374
2,llvm::X86TargetLowering::findRepresentativeClass,160
3,llvm::StringRef::find_first_not_of,63
4,llvm::StringRef::find_last_of,53


>  上面 按 funcName 分组， 不易排查问题，下面 加入 字段  源文件名、函数所在行号、函数所在列号 参与分组， 

> funcName 已经是 函数名 主键了 ，新加字段 都不是 函数名 主键，  理论上不影响结果

In [27]:
# 按照funcEnterId字段分组，并保留组内行数等于1行的组
df_ER_k1__ = df_ER.groupby('funcEnterId').filter(lambda x: len(x) == 1)

# 重置索引，形成新的DataFrame df_ER_k1
df_ER_k1_ = df_ER_k1__.reset_index(drop=True)

# 对df_ER_k1按funcName字段分组，并输出各不同的funcName
只进不出函数名_带函数位置 = df_ER_k1_.groupby( ['funcName','srcFile', 'funcLine', 'funcCol']).size().reset_index()
只进不出函数名_带函数位置.columns = ['funcName', 'srcFile', 'funcLine', 'funcCol', '进入次数']

只进不出函数名_带函数位置_进入次数倒序=只进不出函数名_带函数位置.sort_values(by='进入次数', ascending=False)

pandas.set_option('display.max_rows', None)
pandas.set_option('display.max_colwidth', None)


只进不出函数名_带函数位置_进入次数倒序 = 只进不出函数名_带函数位置_进入次数倒序.reset_index(drop=True)
只进不出函数名_带函数位置_进入次数倒序.shape


(78, 5)

In [28]:

只进不出函数_进入次数求和=只进不出函数名_带函数位置_进入次数倒序.进入次数.sum()

print(f"""
函数进入但无返回次数:{函数进入但无返回次数}, 
只进不出函数_进入次数求和:{只进不出函数_进入次数求和}, 
多出来的次数怎么来的？:{函数进入但无返回次数-只进不出函数_进入次数求和}
""")


函数进入但无返回次数:2073, 
只进不出函数_进入次数求和:1794, 
多出来的次数怎么来的？:279



In [29]:
只进不出函数名_带函数位置_进入次数倒序

,funcName,srcFile,funcLine,funcCol,进入次数
0,clang::IdentifierInfo::getPPKeywordID,/pubx/llvm-project/clang/lib/Basic/IdentifierTable.cpp,324,59,748
1,clang::Preprocessor::HandleDirective,/pubx/llvm-project/clang/lib/Lex/PPDirectives.cpp,1130,51,374
2,llvm::X86TargetLowering::findRepresentativeClass,/pubx/llvm-project/llvm/lib/Target/X86/X86ISelLowering.cpp,2849,58,160
3,llvm::StringRef::find_first_not_of,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,260,70,63
4,llvm::StringRef::find_last_of,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,276,65,53
5,clang::Decl::castFromDeclContext,/pubx/llvm-project/clang/lib/AST/DeclBase.cpp,915,56,52
6,llvm::opt::OptTable::ParseOneArg,/pubx/llvm-project/llvm/lib/Option/OptTable.cpp,404,75,41
7,llvm::MCAssembler::relaxFragment,/pubx/llvm-project/llvm/lib/MC/MCAssembler.cpp,1168,69,39
8,parseSubArch,/pubx/llvm-project/llvm/lib/Support/Triple.cpp,648,64,31
9,llvm::StringRef::find_first_of,/pubx/llvm-project/llvm/lib/Support/StringRef.cpp,235,66,25


#####  1. 无返回 的 函数进入 次数最多的 第一行: 函数 clang::IdentifierInfo::getPPKeywordID

> **由宏调用生成的 带return的case 是没有被插入返回滴答语句的原因**

> **看一下无函数返回 的 函数进入 次数最多的 第一行  ，  其中大量 由宏调用生成的 形如 'case xxx: return yyy;' 的 带return的case**


```cpp
//文件:  /pubx/llvm-project/clang/lib/Basic/IdentifierTable.cpp	
/*第324行*/tok::PPKeywordKind IdentifierInfo::getPPKeywordID() const {

//...

//注意看这个宏CASE定义，形式是 'case xxx: return yyy;'
#define CASE(LEN, FIRST, THIRD, NAME) \
  case HASH(LEN, FIRST, THIRD): \
    return memcmp(Name, #NAME, LEN) ? tok::pp_not_keyword : tok::pp_ ## NAME

  unsigned Len = getLength();
  if (Len < 2) return tok::pp_not_keyword;
  const char *Name = getNameStart();
  switch (HASH(Len, Name[0], Name[2])) {
  default: return tok::pp_not_keyword;
  CASE( 2, 'i', '\0', if);
  CASE( 4, 'e', 'i', elif);
  CASE( 4, 'e', 's', else);
  CASE( 4, 'l', 'n', line);
  CASE( 4, 's', 'c', sccs);
  CASE( 5, 'e', 'd', endif);
  CASE( 5, 'e', 'r', error);
  CASE( 5, 'i', 'e', ident);
  CASE( 5, 'i', 'd', ifdef);
  CASE( 5, 'u', 'd', undef);

  CASE( 6, 'a', 's', assert);
  CASE( 6, 'd', 'f', define);
  CASE( 6, 'i', 'n', ifndef);
  CASE( 6, 'i', 'p', import);
  CASE( 6, 'p', 'a', pragma);

  CASE( 7, 'd', 'f', defined);
  CASE( 7, 'e', 'i', elifdef);
  CASE( 7, 'i', 'c', include);
  CASE( 7, 'w', 'r', warning);

  CASE( 8, 'e', 'i', elifndef);
  CASE( 8, 'u', 'a', unassert);
  CASE(12, 'i', 'c', include_next);

  CASE(14, '_', 'p', __public_macro);

  CASE(15, '_', 'p', __private_macro);

  CASE(16, '_', 'i', __include_macros);
#undef CASE
#undef HASH
  }
}
```



#####  2. 无返回 的 函数进入  第二行: 函数 clang::Preprocessor::HandleDirective	

- **该函数中大量 'case 单return xxx(...)' 已经被花括号包裹，但没有被插入 滴答返回语句，估计是插件libCTk.so的bug**
- 一处: libBrcPlugin.so已经对此  'case  ...; return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
- 两处: libCTk.so居然对case下的if(...){return xxx(...);} 的return前也没有插入滴答返回语句X__funcReturn



```cpp
//文件:  /pubx/llvm-project/clang/lib/Lex/PPDirectives.cpp	
/*第1130行*/void Preprocessor::HandleDirective(Token &Result) {



// Ask what the preprocessor keyword ID is.
/*第1210行*/switch (II->getPPKeywordID()) {
    default:{ 
break;
    }/*加花括号BrcSw【PPDirectives.cpp:1211:12到:1213:5】*/

case tok::pp_if:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIfDirective(Result, SavedHash, ReadAnyTokensBeforeDirective);
    }/*加花括号BrcSw【PPDirectives.cpp:1213:20到:1215:5】*/

case tok::pp_ifdef:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIfdefDirective(Result, SavedHash, false,
                                  true /*not valid for miopt*/);
    }/*加花括号BrcSw【PPDirectives.cpp:1215:23到:1218:5】*/

case tok::pp_ifndef:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIfdefDirective(Result, SavedHash, true,
                                  ReadAnyTokensBeforeDirective);
    }/*加花括号BrcSw【PPDirectives.cpp:1218:24到:1221:5】*/

case tok::pp_elif:
case tok::pp_elifdef:
case tok::pp_elifndef:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleElifFamilyDirective(Result, SavedHash, II->getPPKeywordID());
    }/*加花括号BrcSw【PPDirectives.cpp:1223:26到:1226:5】*/

case tok::pp_else:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleElseDirective(Result, SavedHash);
    }/*加花括号BrcSw【PPDirectives.cpp:1226:22到:1228:5】*/

case tok::pp_endif:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleEndifDirective(Result);

    // C99 6.10.2 - Source File Inclusion.
    }/*加花括号BrcSw【PPDirectives.cpp:1228:23到:1232:5】*/

case tok::pp_include:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      // Handle #include.
      return HandleIncludeDirective(SavedHash.getLocation(), Result);
    }/*加花括号BrcSw【PPDirectives.cpp:1232:25到:1235:5】*/

case tok::pp___include_macros:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      // Handle -imacros.
      return HandleIncludeMacrosDirective(SavedHash.getLocation(), Result);

    // C99 6.10.3 - Macro Replacement.
    }/*加花括号BrcSw【PPDirectives.cpp:1235:34到:1240:5】*/

case tok::pp_define:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleDefineDirective(Result, ImmediatelyAfterTopLevelIfndef);
    }/*加花括号BrcSw【PPDirectives.cpp:1240:24到:1242:5】*/

case tok::pp_undef:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleUndefDirective();

    // C99 6.10.4 - Line Control.
    }/*加花括号BrcSw【PPDirectives.cpp:1242:23到:1246:5】*/

case tok::pp_line:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleLineDirective();

    // C99 6.10.5 - Error Directive.
    }/*加花括号BrcSw【PPDirectives.cpp:1246:22到:1250:5】*/

case tok::pp_error:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleUserDiagnosticDirective(Result, false);

    // C99 6.10.6 - Pragma Directive.
    }/*加花括号BrcSw【PPDirectives.cpp:1250:23到:1254:5】*/

case tok::pp_pragma:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandlePragmaDirective({PIK_HashPragma, SavedHash.getLocation()});

    // GNU Extensions.
    }/*加花括号BrcSw【PPDirectives.cpp:1254:24到:1258:5】*/

case tok::pp_import:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleImportDirective(SavedHash.getLocation(), Result);
    }/*加花括号BrcSw【PPDirectives.cpp:1258:24到:1260:5】*/

case tok::pp_include_next:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIncludeNextDirective(SavedHash.getLocation(), Result);

    }/*加花括号BrcSw【PPDirectives.cpp:1260:30到:1263:5】*/

case tok::pp_warning:{//libBrcPlugin.so已经对此  'case  ...; return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      if (LangOpts.CPlusPlus)
        {Diag(Result, LangOpts.CPlusPlus2b
                         ? diag::warn_cxx2b_compat_warning_directive
                         : diag::ext_pp_warning_directive)
            << /*C++2b*/ 1;}/*BrcThen【PPDirectives.cpp:1265:9到:1268:27】*/
      else
        {Diag(Result, LangOpts.C2x ? diag::warn_c2x_compat_warning_directive
                                  : diag::ext_pp_warning_directive)
            << /*C2x*/ 0;}/*BrcElse【PPDirectives.cpp:1270:9到:1272:25】*/

      return HandleUserDiagnosticDirective(Result, true);
    }/*加花括号BrcSw【PPDirectives.cpp:1263:25到:1275:5】*/

case tok::pp_ident:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIdentSCCSDirective(Result);
    }/*加花括号BrcSw【PPDirectives.cpp:1275:23到:1277:5】*/

case tok::pp_sccs:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      return HandleIdentSCCSDirective(Result);
    }/*加花括号BrcSw【PPDirectives.cpp:1277:22到:1279:5】*/

case tok::pp_assert:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      //isExtension = true;  // FIXME: implement #assert
      break;
    }/*加花括号BrcSw【PPDirectives.cpp:1279:24到:1282:5】*/

case tok::pp_unassert:{//libBrcPlugin.so已经对此  'case 单return xxx(...)'  加了花括号，但libCTk.so没有在此处即return前插入滴答返回语句X__funcReturn
      //isExtension = true;  // FIXME: implement #unassert
      break;

    }/*加花括号BrcSw【PPDirectives.cpp:1282:26到:1286:5】*/

case tok::pp___public_macro:{//libCTk.so居然对case下的if(...){return xxx(...);} 的return前也没有插入滴答返回语句X__funcReturn
      if (getLangOpts().Modules || getLangOpts().ModulesLocalVisibility)
        {return HandleMacroPublicDirective(Result);}/*BrcThen【PPDirectives.cpp:1288:9到:1288:50】*/
      break;

    }/*加花括号BrcSw【PPDirectives.cpp:1286:32到:1291:5】*/

case tok::pp___private_macro:{//libCTk.so居然对case下的if(...){return xxx(...);} 的return前也没有插入滴答返回语句X__funcReturn
      if (getLangOpts().Modules || getLangOpts().ModulesLocalVisibility)
        {return HandleMacroPrivateDirective();}/*BrcThen【PPDirectives.cpp:1293:9到:1293:45】*/
      break;
    }/*加花括号BrcSw【PPDirectives.cpp:1291:33到:1295:5】*/    /*第1295行*/

}





```

#####  3. 无返回 的 函数进入  第三行: 函数 llvm::X86TargetLowering::findRepresentativeClass	

- **default内的 return前没有加 滴答返回语句X__funcReturn**, 有可能是插件libCTk.so的bug


```cpp
std::pair<const TargetRegisterClass *, uint8_t>
X86TargetLowering::findRepresentativeClass(const TargetRegisterInfo *TRI,
                                           MVT VT) const {
  const TargetRegisterClass *RRC = nullptr;
  uint8_t Cost = 1;
  switch (VT.SimpleTy) {

  default:{
    //default内的 return前没有加 滴答返回语句X__funcReturn
    return TargetLowering::findRepresentativeClass(TRI, VT);
  }/*加花括号BrcSw，【X86ISelLowering.cpp:2853:10到:2855:3】*/

case MVT::i8: case MVT::i16: case MVT::i32: case MVT::i64:{
    RRC = Subtarget.is64Bit() ? &X86::GR64RegClass : &X86::GR32RegClass;
    break;
  }/*加花括号BrcSw，【X86ISelLowering.cpp:2855:60到:2858:3】*/

case MVT::x86mmx:{
    RRC = &X86::VR64RegClass;
    break;
  }/*加花括号BrcSw，【X86ISelLowering.cpp:2858:19到:2861:3】*/

case MVT::f32: case MVT::f64:
case MVT::v16i8: case MVT::v8i16: case MVT::v4i32: case MVT::v2i64:
case MVT::v4f32: case MVT::v2f64:
case MVT::v32i8: case MVT::v16i16: case MVT::v8i32: case MVT::v4i64:
case MVT::v8f32: case MVT::v4f64:
case MVT::v64i8: case MVT::v32i16: case MVT::v16i32: case MVT::v8i64:
case MVT::v16f32: case MVT::v8f64:{
    RRC = &X86::VR128XRegClass;
    break;
  }/*加花括号BrcSw，【X86ISelLowering.cpp:2867:36到:2870:3】*/}

  X__funcReturn(&xFuncFrame/*函出*/); 
  return std::make_pair(RRC, Cost);//此return前正确加上 滴答返回语句X__funcReturn 了 
}



```